In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, time
import argparse
import tarfile
import re
import scipy.stats as stats
from astropy import stats as astrostats

##impor  BMD files from directory
##TODO: combine bMD processing to single file, combine LPR processing to single file
import BMD_Analysis_Functions as baf

# Morpho Step 1: Format File

In [2]:
# This is the format_morpho_input function

##############################################
## READ FILE AND SUBSET TO RELEVANT COLUMNS ##
##############################################

# Read morphology file 
df_morph = pd.read_csv('./test_files/7_PAH_zf_morphology_data_2020NOV11_tall_3756.csv', header = 0)

# List relevant column names
relevant_columns = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']

# The input file must absolutely have these columns, no exceptions 
if all(col in df_morph.columns for col in relevant_columns) == False:
    sys.exit(print("The input file", mfiles, "must have the columns:", ', '.join(relevant_columns)))

# Keep only relevant columns
df_morph = df_morph.loc[:,relevant_columns]

##################################
## SUBSET TO RELEVANT ENDPOINTS ##
##################################

# List the relevant endpoints, which is different for BRAIN samples  
if "BRAI" in list(df_morph["endpoint"].unique()):
    relevant_endpoints = ['AXIS', 'BRAI', 'CFIN', 'CIRC', 'DNC_', 'DP24', 'EYE_', 'JAW_', 'MO24', 
                          'MORT', 'NC24', 'NC__', 'OTIC', 'PE__', 'PFIN', 'PIG_', 'SM24', 'SNOU', 
                          'SOMI', 'SWIM', 'TRUN', 'TR__', 'YSE_']
else:
    relevant_endpoints = ['AXIS', 'BRN_', 'CRAN', 'DNC_', 'DP24', 'EDEM', 'LTRK', 'MO24', 'MORT', 
                          'MUSC', 'NC__', 'SKIN','SM24', 'TCHR']

# Get all endpoints 
theEndpoints = df_morph['endpoint'].unique().tolist()

# Get endpoints that are not expected 
unexpected = [end for end in theEndpoints if end not in relevant_endpoints]

# Print a message for missing endpoints 
if (len(unexpected) != 0):
    print("The following endpoints were discovered and are unexpected:", unexpected)

# Subset down to the relevant endpoints 
df_morph = df_morph[df_morph["endpoint"].isin(relevant_endpoints)]

#################################
## CONVERT DO NOT COUNTS TO NA ##
#################################

# If there's a do not count category, remove data 
if ("DNC_" in theEndpoints):

    # Let the user know that data have been removed
    print("A do not count category was detected and those wells were changed to NA.")

    # Wells to remove
    toRmWells = df_morph[(df_morph["endpoint"] == "DNC_") & (df_morph["value"] == 1)]

    # Make the wells NA
    df_morph[(df_morph["chemical.id"].isin(toRmWells["chemical.id"])) &
            (df_morph["conc"].isin(toRmWells["conc"])) &
            (df_morph["plate.id"].isin(toRmWells["plate.id"])) &
            (df_morph["well"].isin(toRmWells["well"]))]["value"] = np.nan

    # And now remove the DNC category entirely
    df_morph = df_morph[df_morph["endpoint"] != "DNC_"]

##################################
## REMOVE MORTALITY FROM COUNTS ##
##################################




A do not count category was detected and those wells were changed to NA.


,chemical.id,conc,endpoint,num.tot,num.nonna,num.affected
0,3756,0.0,ALL_BUT_MORT,36,36,2.0
1,3756,0.0,ANY120,36,36,2.0
2,3756,0.0,ANY24,36,36,4.0
3,3756,0.0,AXIS,36,32,2.0
4,3756,0.0,BRN_,36,32,0.0
...,...,...,...,...,...,...
131,3756,100.0,NC__,36,30,0.0
132,3756,100.0,SKIN,36,30,0.0
133,3756,100.0,SM24,36,32,0.0
134,3756,100.0,TCHR,36,30,0.0


In [ ]:
###########################
## ADD MISSING ENDPOINTS ##
###########################

def new_endpoint(endpoints, new_name):
    """
    Generate a new endpoint which is a binary "or" of other endpoints,
    meaning that if there is a 1 in any of the other endpoints, the 
    resulting endpoint is a 1. Otherwise, it is 0 unless the other 
    endpoints are all NA. Then the final value is NA.
    
    Attributes
    ----
    endpoints: A list of column names, as strings, to binary "or". 
    new_name: The name of the new endpoint. 
    
    """
    sub_df = df_morph[df_morph["endpoint"].isin(endpoints)]
    sub_df["endpoint"] = new_name
    sub_df = sub_df.groupby(by = ["chemical.id", "conc", "plate.id", "well", "endpoint"], as_index = False).sum()
    sub_df['value'].values[sub_df['value'] > 1] = 1 
    return(sub_df)
    
# New endpoints to add is a smaller list if the sample is not from BRAIN
if "BRAI" in list(df_morph["endpoint"].unique()):
    
    df_morph = pd.concat(
        [new_endpoint(['MO24','DP24','SM24','NC24'], 'ANY24'),
         new_endpoint(['MORT', 'YSE_', 'AXIS', 'EYE_', 'SNOU', 'JAW_', 'OTIC', 'PE__', 'BRAI', 
                      'SOMI', 'PFIN', 'CFIN', 'PIG_', 'CIRC', 'TRUN', 'SWIM', 'NC__', 'TR__', 
                      'ANY24'], 'ANY120'),
         new_endpoint(['MO24','MORT'], 'TOT_MORT'),
         new_endpoint(['DP24','SM24','NC24', 'YSE_', 'AXIS', 'EYE_', 'SNOU', 'JAW_', 'OTIC', 'PE__', 
                      'BRAI', 'SOMI', 'PFIN', 'CFIN', 'PIG_', 'CIRC','TRUN', 'SWIM', 'NC__', 'TR__'], 'ALL_BUT_MORT'),
         new_endpoint(['BRAI','OTIC','PFIN'], 'BRN_'),
         new_endpoint(['EYE_', 'SNOU', 'JAW_'], 'CRAN'),
         new_endpoint(['YSE_','PE__'], 'EDEM'),
         new_endpoint(['TRUN','CFIN'], 'LTRK'),
         new_endpoint(['CIRC','SWIM','SOMI'], 'MUSC'),
         new_endpoint(['PIG_'], 'SKIN'),
         new_endpoint(['TR__'], 'TCHR'),
         df_morph]
    )
    
else:
    
    df_morph = pd.concat(

        # 1. Add any effect at 24hrs (combination of MO24, DP24 and SM24) 
        [new_endpoint(['MO24','DP24','SM24'], 'ANY24'),

        # 2. Any effect within 5 days (combination of all measurements at both time points)
        new_endpoint(['AXIS', 'BRN_', 'CRAN', 'EDEM', 'LTRK', 'MORT', 'MUSC', 'NC__', 'SKIN', 'TCHR', 'ANY24'], 'ANY120'),

        # 3. Total mortality (MO24 + MORT) 
        new_endpoint(['MO24','MORT'], 'TOT_MORT'),

        # 4. Any effect except mortality (#2 minus MO24 and MORT)
        new_endpoint(['AXIS', 'BRN_', 'CRAN', 'DP24', 'EDEM', 'LTRK', 'MUSC', 'NC__', 'SKIN', 'SM24', 'TCHR'], 'ALL_BUT_MORT'),
        
        # Add original dataframe
        df_morph]
    )
    
###########################################
## CALCULATE VARIABLES FOR DOSE RESPONSE ##
###########################################

# Create groups of each chemical id, concentration, and plate id
plate_groups = df_morph.drop(["well"], 1).groupby(by = ["chemical.id", "conc", "plate.id", "endpoint"], as_index = False)

# Get the number of samples per group
num_tot_samples = plate_groups.size().rename(columns = {"size": "num.tot"})

# Get the number of non-na samples per groups
num_nonna = plate_groups.count().rename(columns = {"value": "num.nonna"})

# Get the number affected
num_affected = plate_groups.sum().rename(columns = {"value": "num.affected"})

# Merge to create missingness dataframe
plate_groups = pd.merge(pd.merge(num_tot_samples, num_nonna), num_affected)

# Create IDs of chemical.id, plate.id, and endpoint in plate_groups 
ids = []
for row in range(len(plate_groups)):
    ids.append(str(plate_groups["chemical.id"][row]) + " " + str(plate_groups["plate.id"][row]) + " " + str(plate_groups["endpoint"][row]))
plate_groups["ids"] = ids

#####################################################################
## REMOVE VARIABLES WITH HIGH MISSINGNESS IN BASELINE MEASUREMENTS ##
#####################################################################

# Identify 0 (baseline) concentrations with high missingness (greater than 50% missing or less than 50% non-missing)
missingness = plate_groups.loc[plate_groups["conc"] == 0]
missingness["keep"] = missingness["num.nonna"] / missingness["num.tot"] > 0.5 # TODO: Add a report of what was removed --> txt file "nothing removed"

# Identify plates to keep 
tokeep = missingness.loc[missingness["keep"]]["ids"].tolist()
plate_groups = plate_groups[plate_groups["ids"].isin(tokeep)]

# Stop if everything gets removed
if len(plate_groups) == 0:
    sys.exit("Everything was removed with the 50% missingness filter")

#######################################
## REGROUP WITHOUT PLATE IDS AND SUM ##
#######################################

# First, remove plate.id and ids column
chemical_groups = plate_groups.drop(columns = ["plate.id", "ids"])

# Group by chemical.id, concentration, and endpoint. Then, sum the results. 
chemical_groups = chemical_groups.groupby(by = ["chemical.id", "conc", "endpoint"]).sum().reset_index()

############################
## RETURN FORMATTED TABLE ##
############################
chemical_groups

# Morpho Step 2: Generate Dose Response and Write Flags

In [47]:
# This is the generate_dose_response function

dose_response = chemical_groups

'''This function performs feasibility analysis
for dose response data. The value returned is a
flag indicating data quality as defined below:

0: Not enough dose groups for BMD analysis. BMD analysis not performed. BAD.
1: No trend detected in dose-response data. BMD analysis not performed. BAD. 
2: Good dose-response data. BMD analysis is performed. GOOD.
3: Dose-response data quality poor. BMD analysis might be unreliable. GOOD. 
4: Data resolution poor. BMD analysis might be unreliable. MODERATE.
5: Negative correlation. 

'''

########################
## CALCULATE RESPONSE ##
########################

# Add an id column
ids = []
for row in range(len(dose_response)):
    ids.append(str(dose_response["chemical.id"][row]) + " " + str(dose_response["endpoint"][row]))
dose_response["ids"] = ids

# Response is the number affected over the number of embryos (non-na)
dose_response["frac.affected"] = dose_response["num.affected"] / dose_response["num.nonna"]

####################################
## GENERATE QUALITY CONTROL FLAGS ##
####################################

# Count the number of unique concentrations per chemical id and endpoint pairing 
BMD_Flags = dose_response.groupby(["chemical.id", "endpoint", "ids"])["conc"].nunique().reset_index().rename(columns = {"conc": "num.conc"})

# Add a flag category
BMD_Flags["flag"] = None

# If there are less than 3 points, the BMD flag is 0 - not enough dose groups
BMD_Flags["flag"].values[BMD_Flags["num.conc"] < 3] = 0

# Change dose response 
dose_response["conc"] = dose_response["conc"].astype('float') + 1e-15

# Calculate the spearman correlation
spear = dose_response[["chemical.id", "endpoint", "conc", "frac.affected"]].groupby(["chemical.id", "endpoint"]).corr(method = "spearman").unstack().iloc[:,1].reset_index()
spear = spear.set_axis(["chemical.id", "endpoint", "spearman"], axis = 1)

# Merge spearman to the BMD_Flags dataframe
BMD_Flags = BMD_Flags.merge(spear)

# If spearman correlation is below 0.2 or is NaN, the Flag is 1 - no strong trend 
BMD_Flags["flag"].values[(BMD_Flags["spearman"] < 0.2) | (BMD_Flags["spearman"].isna())] = 1

# If the correlation is above 0.25 or below 0.8, run a t-test
BMD_Flags["run.ttest"] = (BMD_Flags["spearman"] >= 0.20) & (BMD_Flags["spearman"] <= 0.80)

# If the correlation is above 0.8, assign the flag at 2 - good
BMD_Flags["flag"].values[BMD_Flags["spearman"] > 0.8] = 2

# Run the t-test only where indicated 
ttest = dose_response[dose_response["ids"].isin(BMD_Flags[BMD_Flags["run.ttest"]]["ids"].to_list())][["ids", "frac.affected"]]
ttest = ttest.groupby("ids").apply(lambda df: stats.ttest_1samp(np.diff(df["frac.affected"]), 0)[1]).reset_index().rename(columns = {0:"ttest.pval"})

# Merge ttest results 
BMD_Flags = BMD_Flags.merge(ttest, on = "ids", how = "outer")

# A p-value of less than 0.05 gets a flag of 2 - good, from 0.05 to 0.32 gets a 3 - unreliable, 
# and greater than 0.32 gets very unreliable. 
BMD_Flags["flag"].values[BMD_Flags["ttest.pval"] <= 0.05] = 2
BMD_Flags["flag"].values[(BMD_Flags["ttest.pval"] > 0.05) & (BMD_Flags["ttest.pval"] <= 0.32)] = 3
BMD_Flags["flag"].values[BMD_Flags["ttest.pval"] > 0.32] = 4

BMD_Flags


,chemical.id,endpoint,ids,num.conc,flag,spearman,run.ttest,ttest.pval
0,3756,ALL_BUT_MORT,3756 ALL_BUT_MORT,8,4,0.739516,True,0.852840
1,3756,ANY120,3756 ANY120,8,4,0.566306,True,0.896837
2,3756,ANY24,3756 ANY24,8,4,0.727393,True,0.792745
3,3756,AXIS,3756 AXIS,8,4,0.357143,True,0.900140
4,3756,BRN_,3756 BRN_,8,1,NaN,False,NaN
5,3756,CRAN,3756 CRAN,8,4,0.357143,True,0.900140
6,3756,DP24,3756 DP24,8,2,0.805118,False,NaN
7,3756,EDEM,3756 EDEM,8,4,0.357143,True,0.900140
8,3756,LTRK,3756 LTRK,8,1,0.000000,False,NaN
9,3756,MO24,3756 MO24,8,4,0.578355,True,1.000000


# Morpho Step 3: Select and Run Models

In [117]:
###########################################
## CALCULATE VALUES FOR LOW QUALITY DATA ##
###########################################

low_quality = dose_response[dose_response["ids"].isin(BMD_Flags[BMD_Flags["flag"].isin([0,1])]["ids"])].groupby("ids")

# Calculate low quality metrics and start new BMDS file 
BMDS_LowQual = low_quality.apply(lambda df: np.trapz(df["frac.affected"], x = df["conc"])).reset_index().rename(columns = {0: "AUC"})
BMDS_LowQual[["Model", "BMD10", "BMDL", "BMD50"]] = np.nan
BMDS_LowQual["Min_Dose"] = round(low_quality[["ids", "conc"]].min("conc").reset_index()["conc"], 4)
BMDS_LowQual["Max_Dose"] = round(low_quality[["ids", "conc"]].max("conc").reset_index()["conc"], 4)
BMDS_LowQual["AUC_Norm"] = BMDS_LowQual["AUC"] / (BMDS_LowQual["Max_Dose"] - BMDS_LowQual["Min_Dose"])

# Order columns
BMDS_LowQual = BMDS_LowQual[["ids", "Model", "BMD10", "BMDL", "BMD50", "AUC", "Min_Dose", "Max_Dose", "AUC_Norm"]]

################
## RUN MODELS ##
################

# Define a function to run models
def select_and_run_models(ID):
    '''For each ID, run the 8 regression models and return the best one.'''
    
    Data = dose_response[dose_response["ids"] == ID]

    # Get the number of nonNA samples per dose
    NonNATotals = Data["num.nonna"].tolist() 

    # Regression model function
    def run_regression_model(modelfun, fittedfun):
        '''Fit the regression model and return the parameters, fitted_values, and the p_value'''

        # Run the model
        model = modelfun(Data[["conc", "num.affected", "num.nonna"]].astype('float').copy())

        # Get the model parameters
        model_params = model.fit().params

        # Get the model's fitted values
        model_fittedvals = fittedfun(Data["conc"], model_params)

        # Get the p_value
        model_pval = calc_p_value(model_fittedvals, model_params)

        # Get the AIC
        AIC = -2*model.fit().llf + (2 * len(model_params))

        # Return a list
        return([model, model_params, model_fittedvals, model_pval, AIC])

    # Calculate P-Value Function
    def calc_p_value(PredictedValues, Params):
        '''Return a p-value of model fit for each unique ID and Model dataframe pairing'''

        # Get the experimental values 
        ExperimentalValues = Data["frac.affected"].tolist()

        # Now, calculate the chi squared value
        ChiSquared = ((NonNATotals / (PredictedValues * (1 - PredictedValues))) * (ExperimentalValues - PredictedValues)**2).sum()

        # Calculate a p-value of fit 
        return(1 - stats.chi2.sf(ChiSquared, len(NonNATotals) - len(Params)))

    # Run regression models in a dictionary
    models = {

        ## Logistic ##
        "Logistic": run_regression_model(baf.Logistic, baf.logistic_fun),

        ## Gamma ## 
        "Gamma": run_regression_model(baf.Gamma, baf.gamma_fun),

        ## Weibull ##
        "Weibull": run_regression_model(baf.Weibull, baf.weibull_fun),

        ## Log-Logistic ##
        "Log Logistic": run_regression_model(baf.Log_Logistic, baf.log_logistic_fun),

        ## Probit ##
        "Probit": run_regression_model(baf.Probit, baf.probit_fun),

        ## Log-Probit ##
        "Log Probit": run_regression_model(baf.Log_Probit, baf.log_probit_fun),

        ## Multistage ##
        "Multistage": run_regression_model(baf.Multistage_2, baf.multistage_2_fun),

        ## Quantal Linear ##
        "Quantal Linear": run_regression_model(baf.Quantal_Linear, baf.quantal_linear_fun),

    }

    # Iterate through all p-values 
    p_values = {}
    for key in models.keys():
        p_values[key] = models[key][3]

    # Iterate through all AICs 
    aics = {}
    for key in models.keys():
        aics[key] = models[key][4]

    # Determine the best model
    BestModel = min(aics, key=lambda k: aics[k]) 

    # Return results 
    return(
        [
            p_values,
            models[BestModel],
            BestModel,
            aics
        ]
    )

# Run the models 
model_results = {}
for id in BMD_Flags[BMD_Flags["flag"].isin([2,3,4,5])]["ids"]:
    model_results[id] = select_and_run_models(id)

# Make p-value dataframe
p_value_list = []
for id in model_results.keys():
    theDict = model_results[id][0]
    theDict["ids"] = id
    p_value_list.append(theDict)

p_value_df = pd.DataFrame(p_value_list)

########################################
## CALCULATE FLAGS FOR MODEL MATCHING ##
########################################

# Get flags of whether p_values are all below 0.1
Analysis_Flags = p_value_df
Analysis_Flags.iloc[:,:8] = Analysis_Flags.iloc[:,:8] < 0.1
Analysis_Flags = Analysis_Flags.groupby("ids").apply(lambda df: int(df.all(axis = 1))).reset_index().rename(columns = {0:"BMD_Analysis_Flag"})
BMD_Flags = pd.merge(BMD_Flags, Analysis_Flags)

#######################################
## PULL AKAIKE INFORMATION CRITERION ##
#######################################

# Make aic dataframe
aic_list = []
for id in model_results.keys():
    theDict = model_results[id][3]
    theDict["ids"] = id
    aic_list.append(theDict)

aic_df = pd.DataFrame(aic_list)

##############################
## CALCULATE BENCHMARK DOSE ##
##############################

def Calculate_BMD(Model, params, BenchmarkResponse = 0.1):
    '''Calculate a benchmark dose'''

    # For each model, extract the parameters and run the calculations
    if (Model == "Logistic"): 
        alpha_, beta_ = params
        return(np.log((1 + np.exp(-alpha_)*BenchmarkResponse)/(1-BenchmarkResponse))/beta_)
    elif (Model == "Gamma"):
        g_, alpha_, beta_ = params
        return(stats.gamma.ppf(BenchmarkResponse, alpha_, scale = 1/beta_))
    elif (Model == "Weibull"):
        g_, alpha_, beta_ = params
        return((-np.log(1 - BenchmarkResponse)/beta_)**(1/alpha_))
    elif (Model == "Log Logistic"):
        g_, alpha_, beta_ = params
        return(np.exp((np.log(BenchmarkResponse/(1-BenchmarkResponse)) - alpha_)/beta_))
    elif (Model == "Probit"):
        alpha_, beta_ = params
        p_0 = stats.norm.cdf(alpha_)
        p_BMD = p_0 + (1 - p_0)*BenchmarkResponse
        return((stats.norm.ppf(p_BMD) - alpha_)/beta_)
    elif (Model == "Log Probit"):
        g_, alpha_, beta_ = params
        return(np.exp((stats.norm.ppf(BenchmarkResponse) - alpha_)/beta_))
    elif (Model == "Multistage"):
        g_, alpha_, beta_ = params
        return((-beta_ + np.sqrt((beta_**2) - (4*beta2_*np.log(1 - BMR))))/(2*beta2_))
    elif (Model == "Quantal Linear"):
        g_, beta_ = params
        return(-np.log(1 - BenchmarkResponse)/beta_)
    else:
        print(Model, "was not recognized as an acceptable model choice.")

def Calculate_BMDL(Model, FittedModelObj, Data, BMD10, params, MaxIterations = 100, ToleranceThreshold = 1e-4):
    '''Run BMDL Function Test'''

    # Reformat data
    Data = Data[["conc", "num.affected", "num.nonna"]].astype('float').copy()

    # Define an initial low and high threshold
    BMD_Low = BMD10/10
    BMD_High = BMD10

    # Start a counter and set tolerance to 1
    Iteration_Count = 0
    Tolerance = 1

    # Set a LLV Threhold
    BMDL_LLV_Thresh = FittedModelObj.fit().llf - stats.chi2.ppf(0.9, 1)/2

    # Start a while condition loop
    while ((Tolerance > ToleranceThreshold) and (Iteration_Count <= MaxIterations)):
        
        # Add to the iteration counters 
        Iteration_Count+=1

        # If maximum iterations are reached, set BMDL to NA and break
        if (Iteration_Count == MaxIterations):
            BMDL = np.nan
            break

        # BMDL should be the mid value between the low and high estimate
        BMDL = (BMD_Low + BMD_High)/2
        ModelObj = np.nan

        # Select the correct BMD model
        if (Model == "Logistic"):
            ModelObj = baf.Logistic_BMD(Data).profile_ll_fit([params[0], BMDL]) # Value is alpha
        elif (Model == "Gamma"):
            ModelObj = baf.Gamma_BMD(Data).profile_ll_fit([params[0], params[1], BMDL]) # Value is g and alpha
        elif (Model == "Weibull"):
            ModelObj = baf.Weibull_BMD(Data).profile_ll_fit([params[0], params[1], BMDL]) # Value is g and alpha
        elif (Model == "Log Logistic"):
            ModelObj = baf.Log_Logistic_BMD(Data).profile_ll_fit([params[0], params[2], BMDL]) # Value is g and beta
        elif (Model == "Probit"):
            ModelObj = baf.Probit_BMD(Data).profile_ll_fit([params[0], BMDL]) # Value is alpha
        elif (Model == "Log Probit"):
            ModelObj = baf.Log_Probit_BMD(Data).profile_ll_fit([params[0], params[1], BMDL]) # Value is g and alpha
        elif (Model == "Multistage"):
            ModelObj = baf.Multistage_2_BMD(Data).profile_ll_fit([params[0], params[1], BMDL]) # Value is g and beta 1
        elif (Model == "Quantal Linear"):
            ModelObj = baf.Quantal_Linear_BMD(Data).profile_ll_fit([params[0], BMDL]) # Value is g
        else:
            print(Model, "was not recognized as an acceptable model choice.")

        # Pull the llf 
        LLF = ModelObj.llf

        # If the calculated LLF is not within the threshold, set high to BMDL and run again 
        if((LLF - BMDL_LLV_Thresh) > 0):
            BMD_High = BMDL
        # Otherwise, set low to BMDL
        else:
            BMD_Low = BMDL

        Tolerance = abs(LLF - BMDL_LLV_Thresh)

    return(BMDL)


# Build BMD table for fitted data 
BMDS_Model = []

for id in model_results.keys():

    # Get the model name 
    Model = model_results[id][2]

    # Get the fitted modeol object
    FittedModelObj = model_results[id][1][0]

    # Get the parameters 
    params = model_results[id][1][1]

    # Get the BMD10 value 
    BMD10 = Calculate_BMD(Model, params, 0.1)

    # Get the dose response data
    Data = dose_response[dose_response["ids"] == id]

    # Get the AUC, min, and max dose 
    AUC = np.trapz(Data["frac.affected"], x = Data["conc"])
    Min_Dose = round(min(Data["conc"]), 4)
    Max_Dose = round(max(Data["conc"]), 4)

    # Return results in a dictionary
    rowDict = {
        "ids": id,
        "Model": Model,
        "BMD10": BMD10, 
        "BMDL": Calculate_BMDL(Model, FittedModelObj, Data, BMD10, params),
        "BMD50": Calculate_BMD(Model, params, 0.5),
        "AUC": AUC,
        "Min_Dose": Min_Dose,
        "Max_Dose": Max_Dose,
        "AUC_Norm": AUC / (Max_Dose - Min_Dose)
    }
    BMDS_Model.append(rowDict)

BMDS_Model = pd.DataFrame(BMDS_Model)
BMDS_Model

ValueError: cannot insert ids, already exists

,ids,Model,BMD10,BMDL,BMD50,AUC,Min_Dose,Max_Dose,AUC_Norm
0,3756 ALL_BUT_MORT,Quantal Linear,100.870848,52.520072,663.610495,9.617500,0.0,100.0,0.096175
1,3756 ANY120,Log Logistic,36.500441,NaN,14600.985783,15.625278,0.0,100.0,0.156253
2,3756 ANY24,Quantal Linear,57.430877,34.438108,377.827027,15.699722,0.0,100.0,0.156997
3,3756 AXIS,Quantal Linear,153.476832,62.585087,1009.695452,8.139358,0.0,100.0,0.081394
4,3756 CRAN,Quantal Linear,153.476832,62.585087,1009.695452,8.139358,0.0,100.0,0.081394
5,3756 DP24,Quantal Linear,107.930739,61.374738,710.056198,5.830038,0.0,100.0,0.058300
6,3756 EDEM,Quantal Linear,153.476832,62.585087,1009.695452,8.139358,0.0,100.0,0.081394
7,3756 MO24,Quantal Linear,125.579426,56.662121,826.163621,10.764444,0.0,100.0,0.107644
8,3756 MORT,Log Logistic,45.852837,4.742707,445811.848379,10.323276,0.0,100.0,0.103233
9,3756 MUSC,Quantal Linear,153.476832,62.585087,1009.695452,8.139358,0.0,100.0,0.081394


# Export: New BMDS

In [116]:
def BMD_Range_Flag(id, BMD):
    '''Determine if a BMD value is within the reasonable range'''
    
    # Get concentrations
    concs = dose_response[dose_response["ids"] == id]["conc"]

    if (np.isnan(BMD)):
        return(np.nan)
    elif (BMD <= max(concs) and BMD >= min(concs)):
        return(0)
    else:
        return(1)

# Start final BMDS data frame 
BMDS_Final = pd.concat([BMDS_LowQual, BMDS_Model]).merge(BMD_Flags[["ids", "flag", "BMD_Analysis_Flag"]], on = "ids", how = "left").rename(columns = {"flag":"DataQC_Flag"})

# Change NaN data QC flag to 1
BMDS_Final["DataQC_Flag"][pd.isnull(BMDS_Final["DataQC_Flag"])] = 1

# Add BMD10 and BMD50 flags
BMDS_Final["BMD10_Flag"] = [BMD_Range_Flag(BMDS_Final["ids"][pos], BMDS_Final["BMD10"][pos]) for pos in range(len(BMDS_Final))]
BMDS_Final["BMD50_Flag"] = [BMD_Range_Flag(BMDS_Final["ids"][pos], BMDS_Final["BMD50"][pos]) for pos in range(len(BMDS_Final))]

# Add columns for printing
BMDS_Final["Chemical_ID"] = [x.split(" ")[0] for x in BMDS_Final["ids"].to_list()]
BMDS_Final["End_Point"] = [x.split(" ")[1] for x in BMDS_Final["ids"].to_list()]

BMDS_Final = BMDS_Final[["Chemical_ID", "End_Point", "Model", "BMD10", "BMDL", "BMD50", "AUC", "Min_Dose", "Max_Dose", "AUC_Norm", 
            "DataQC_Flag", "BMD_Analysis_Flag", "BMD10_Flag", "BMD50_Flag", "ids"]]
BMDS_Final

,Chemical_ID,End_Point,Model,BMD10,BMDL,BMD50,AUC,Min_Dose,Max_Dose,AUC_Norm,DataQC_Flag,BMD_Analysis_Flag,BMD10_Flag,BMD50_Flag,ids
0,3756,BRN_,NaN,NaN,NaN,NaN,0.000000,0.0,100.0,0.000000,1,NaN,NaN,NaN,3756 BRN_
1,3756,LTRK,NaN,NaN,NaN,NaN,2.699267,0.0,100.0,0.026993,1,NaN,NaN,NaN,3756 LTRK
2,3756,NC__,NaN,NaN,NaN,NaN,0.000000,0.0,100.0,0.000000,1,NaN,NaN,NaN,3756 NC__
3,3756,SKIN,NaN,NaN,NaN,NaN,0.000000,0.0,100.0,0.000000,1,NaN,NaN,NaN,3756 SKIN
4,3756,SM24,NaN,NaN,NaN,NaN,0.000000,0.0,100.0,0.000000,1,NaN,NaN,NaN,3756 SM24
5,3756,ALL_BUT_MORT,Quantal Linear,100.870848,52.520072,663.610495,9.617500,0.0,100.0,0.096175,4,1.0,1.0,1.0,3756 ALL_BUT_MORT
6,3756,ANY120,Log Logistic,36.500441,NaN,14600.985783,15.625278,0.0,100.0,0.156253,4,1.0,0.0,1.0,3756 ANY120
7,3756,ANY24,Quantal Linear,57.430877,34.438108,377.827027,15.699722,0.0,100.0,0.156997,4,1.0,0.0,1.0,3756 ANY24
8,3756,AXIS,Quantal Linear,153.476832,62.585087,1009.695452,8.139358,0.0,100.0,0.081394,4,1.0,1.0,1.0,3756 AXIS
9,3756,CRAN,Quantal Linear,153.476832,62.585087,1009.695452,8.139358,0.0,100.0,0.081394,4,1.0,1.0,1.0,3756 CRAN


# Export: New Fits

In [53]:
def calc_fits(ID):
    '''Calculate the fitted values based on the ID, using the model_results and the final BMDS table.'''

    # If the ID is not found in the model_results, then return blanks for x and y 
    if ((ID in model_results) == False):
        return({
            "Chemical_ID": ID.split(" ")[0],
            "End_Point": ID.split(" ")[1],
            "X_vals": np.nan,
            "Y_vals": np.nan
        })

    def gen_uneven_spacing(doses, int_steps = 10):
        '''Generates ten steps of points between measurements'''
        dose_samples = list()
        for dose_index in range(len(doses) - 1):
            dose_samples.extend(np.linspace(doses[dose_index],doses[dose_index + 1], int_steps).tolist())
        return np.unique(dose_samples)


    # Get the model
    model = model_results[ID][2]

    # Get the parameters
    params = model_results[ID][1][1]

    # Define the uneven x values
    dose_x_vals = np.round(gen_uneven_spacing(dose_response[dose_response["ids"] == ID]["conc"].to_list()), 4)

    def run_fitted_model(fittedfun, dose_x_vals = dose_x_vals, params = params):
        '''Run modeled x values through the fit function'''
        return(fittedfun(dose_x_vals, params))

    # Define a y value holder
    dose_y_vals = np.nan

    # Get the y values 
    if (Model == "Logistic"):
        dose_y_vals = run_fitted_model(baf.logistic_fun)
    elif (Model == "Gamma"):
        dose_y_vals = run_fitted_model(baf.gamma_fun)
    elif (Model == "Weibull"):
        dose_y_vals = run_fitted_model(baf.weibull_fun)
    elif (Model == "Log Logistic"):
        dose_y_vals = run_fitted_model(baf.log_logistic_fun)
    elif (Model == "Probit"):
        dose_y_vals = run_fitted_model(baf.probit_fun)
    elif (Model == "Log Probit"):
        dose_y_vals = run_fitted_model(baf.log_probit_fun)
    elif (Model == "Multistage"):
        dose_y_vals = run_fitted_model(baf.multistage_2_fun)
    elif (Model == "Quantal Linear"):
        dose_y_vals = run_fitted_model(baf.quantal_linear_fun)

    return({
        "Chemical_ID": [ID.split(" ")[0]] * len(dose_x_vals),
        "End_Point": [ID.split(" ")[1]] * len(dose_x_vals),
        "X_vals": dose_x_vals,
        "Y_vals": np.round(dose_y_vals, 8)
    })

Fits_List = []
for ID in BMDS_Final["ids"]:
    Fits_List.append(calc_fits(ID))

Fits_Final = pd.DataFrame(Fits_List).explode(["Chemical_ID", "End_Point", "X_vals", "Y_vals"])
Fits_Final


NameError: name 'BMDS_Final' is not defined

# Export: New Dose

In [ ]:
# Select required columns and rename 
Dose_Final = dose_response[["chemical.id", "endpoint", "conc", "frac.affected", "num.affected", "num.nonna", "ids"]].rename(columns = {"chemical.id":"Chemical_ID", "endpoint":"End_Point","conc":"Dose","frac.affected":"Response"})

# Round dose to 4 decimal points, and endpoint to 8
Dose_Final["Dose"] = round(Dose_Final["Dose"], 4)
Dose_Final["Response"] = round(Dose_Final["Response"], 8)

# Calculate confidence intervals 
CI = pd.DataFrame([np.abs(astrostats.binom_conf_interval(Dose_Final["num.affected"][row], Dose_Final["num.nonna"][row], confidence_level = 0.95) - Dose_Final["Response"][row]) for row in range(len(Dose_Final))]).rename(columns = {0:"CI_Lo", 1:"CI_Hi"})
CI["CI_Lo"] = round(CI["CI_Lo"], 6)
CI["CI_Hi"] = round(CI["CI_Hi"], 6)

# Set the range of possible values between 0 and 1
Dose_Final = pd.concat([Dose_Final, CI], axis = 1)
Dose_Final

,Chemical_ID,End_Point,Dose,Response,num.affected,num.nonna,ids,CI_Lo,CI_Hi
0,3756,ALL_BUT_MORT,0.0,0.055556,2.0,36,3756 ALL_BUT_MORT,0.040186,0.125891
1,3756,ANY120,0.0,0.055556,2.0,36,3756 ANY120,0.040186,0.125891
2,3756,ANY24,0.0,0.111111,4.0,36,3756 ANY24,0.067045,0.142037
3,3756,AXIS,0.0,0.062500,2.0,32,3756 AXIS,0.045189,0.138971
4,3756,BRN_,0.0,0.000000,0.0,32,3756 BRN_,0.000000,0.107179
...,...,...,...,...,...,...,...,...,...
139,3756,NC__,100.0,0.000000,0.0,30,3756 NC__,0.000000,0.113513
140,3756,SKIN,100.0,0.000000,0.0,30,3756 SKIN,0.000000,0.113513
141,3756,SM24,100.0,0.000000,0.0,32,3756 SM24,0.000000,0.107179
142,3756,TCHR,100.0,0.000000,0.0,30,3756 TCHR,0.000000,0.113513


# Filter by LPR

In [ ]:
df_morph

,chemical.id,conc,plate.id,well,endpoint,value
0,3756,0.0,20544,H01,ANY24,0.0
1,3756,0.0,20544,H02,ANY24,1.0
2,3756,0.0,20544,H03,ANY24,0.0
3,3756,0.0,20544,H04,ANY24,0.0
4,3756,0.0,20544,H05,ANY24,0.0
...,...,...,...,...,...,...
4027,3756,100.0,20625,A08,DNC_,0.0
4028,3756,100.0,20625,A09,DNC_,0.0
4029,3756,100.0,20625,A10,DNC_,0.0
4030,3756,100.0,20625,A11,DNC_,0.0


In [ ]:
# Read LPR data
df_LPR = pd.read_csv('./test_files/7_PAH_zf_LPR_data_2021JAN11_3756.csv', header = 0)

# Drop NA
df_LPR = df_LPR.dropna().rename(columns = {"variable":"timepoint"}).loc[:,['chemical.id', 'conc', 'plate.id', 'well', 'timepoint', 'value']]

df_LPR

,chemical.id,conc,plate.id,well,timepoint,value
0,3756,0.0,20544,H01,t0,0.000
3,3756,0.0,20544,H04,t0,1.083
4,3756,0.0,20544,H05,t0,1.608
5,3756,0.0,20544,H06,t0,4.979
6,3756,0.0,20544,H07,t0,2.394
...,...,...,...,...,...,...
69115,3756,100.0,20625,A08,t239,0.000
69116,3756,100.0,20625,A09,t239,0.000
69117,3756,100.0,20625,A10,t239,0.000
69118,3756,100.0,20625,A11,t239,8.332


In [ ]:

df_morph[df_morph["endpoint"] == "MORT"].dropna().query("value == 0").drop(["endpoint", "value"], 1)

,chemical.id,conc,plate.id,well
864,3756,0.0,20544,H01
866,3756,0.0,20544,H03
867,3756,0.0,20544,H04
868,3756,0.0,20544,H05
869,3756,0.0,20544,H06
...,...,...,...,...
1147,3756,100.0,20625,A08
1148,3756,100.0,20625,A09
1149,3756,100.0,20625,A10
1150,3756,100.0,20625,A11
